In [3]:
x   = SR.var('x')
x0  = 0
f   = sin(x) * e^(-x)
p   = plot(f, -1, 5, thickness=2)
dot = point((x0, f(x=x0)), pointsize=80, rgbcolor=(1, 0, 0))

@interact
def _(order=slider([1 .. 12])):
  ft = f.taylor(x, x0, order)
  pt = plot(ft, -1, 5, color='green', thickness=2)
  pretty_print(html(r'$f(x)\;=\;%s$' % latex(f)))
  pretty_print(html(r'$\hat{f}(x;%s)\;=\;%s+\mathcal{O}(x^{%s})$' % (x0, latex(ft), order+1)))
  show(dot + p + pt, ymin=-.5, ymax=1)

Interactive function <function _ at 0x7f3844d0a8c0> with 1 widget
  order: SelectionSlider(value=1, options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), description='order')

In [3]:
U13 = matroids.Uniform(1,3)
U23 = matroids.Uniform(2,3)
U33 = matroids.Uniform(3,3)

In [8]:
P1 = U13.matroid_polytope()
P2 = U23.matroid_polytope()
P3 = U33.matroid_polytope()

A 2-dimensional polyhedron in ??^3 defined as the convex hull of 6 vertices..
$x_1+x_2+x_3=\begin{pmatrix} 3+1\\2\end{pmatrix}$

In [7]:
F=P1+P2+P3
F.plot(opacity=0.6)

Graphics3d Object

In [9]:
from collections import defaultdict
var('t')
a = 0; b= 2*pi

def random_line3d(bound):
    '''Random line in R^3: first choose the guiding vector of the line,
    then choose a point in the plane p perpendicular to that vector.
    '''
    p = vector([(2*random() - 1) for _ in range(3)])
    v = p/norm(p)
    v2, v3 = matrix(v).right_kernel().basis()
    if det(matrix([v, v2, v3]))<0:
        v2, v3 = v3, v2
    v2 = v2/norm(v2)
    v3 = v3 - (v3*v2)*v2
    v3 = v3/norm(v3)
    return v, (2*random()*bound - bound, v2), (2*random()*bound - bound, v3)

def winding_number(x, y, x0, y0):
    x -= x0
    y -= y0
    r2 = x^2 + y^2
    xp = x.derivative(t)
    yp = y.derivative(t)
    integrando(t) = (x*yp -y*xp)/r2
    i,e = numerical_integral(integrando, a, b)
    return round(i/(2*pi))

def linking_number(curve, line):
    v, (a2, v2), (a3, v3) = line
    M = matrix([v, v2, v3])
#    curve2d = (curve*M.inverse())[1:] #This is VERY slow, for some reason!
    curve2d = sum(c*v for c,v in zip(curve,M.inverse()))[1:]
    x, y = curve2d
    return winding_number(x, y, a2,a3)

def color_ln(number):
    if number:
        return (0,1-1/(1+number),0)
    else:
        return (1,0,0)

def banchoff_pohl(curve, L, M):
    ln_d = defaultdict(int)
    pp = parametric_plot3d(curve, (t,0,2*pi), thickness=2)
    for k in range(L):
        a_line = random_line3d(M)
        ln = abs(linking_number(curve, a_line))
        v, (l1, v1), (l2, v2) = a_line
        pp += parametric_plot3d(l1*v1+l2*v2+t*v,(t,-2,2), 
                                color=color_ln(ln))
        ln_d[ln] += 1
    return ln_d, pp

def print_stats(d):
    print('Number of lines with linking number k:')
    print(', '.join('%d:%d' % kv for kv in d.items()))

@interact
def bp_interact( u1 = text_control('x, y, z coordinates of a closed space curve in [0,2*pi]'),
                 curvax = input_box(cos(t), label='x(t)' ),
                 curvay = input_box(sin(t), label='y(t)' ),
                 curvaz = input_box(0, label='y(t)' ),
                 u2 = text_control('The curve should be contained in a ball of radius M'),
                 M  = 1,
                 u3 = text_control('Use L lines chosen randomly'),
                 L  = 10):
    ln_d, p = banchoff_pohl(vector((curvax, curvay, curvaz)), L, M)
    p.show(aspect_ratio=1, xmin=-2, xmax=2, ymin=-2,ymax=2)
    bp_area_aprox = (sum(k^2*v for k,v in ln_d.iteritems())/L)*2*pi*M^2
    print('Bahnchoff-Pohl area of the curve(aprox): %f' % bp_area_aprox)
    print_stats(ln_d)

Graphics3d Object

AttributeError: 'collections.defaultdict' object has no attribute 'iteritems'